## Smith-Waterman Algorithm with Affine Scoring

Affine scoring gives a higher penalty for the *first gap* in a sequence of gaps, this is because when there is a gap in an alignment what is more important usually is that the fact that there is gap, particular size of it might come secondary.

$ \text{Gap penalty} = (length_{gap} \cdot e) + d $ where d is the gap **initialization penalty** and e is the **extension penalty**.

#### Some definitions:
$$
\begin{split}
s(a,b) & = \text{score of aligning a and b} \\
F(i,j) & = \text{optimal alignment between substrings $A(1:i)$ and $B(1:j)$}
\end{split}
$$

#### Initialization:
$$
\begin{split}
F(i,0) = 0 \\
F(0,j) = 0 
\end{split}
$$
#### Induction:
$$
\begin{split}
F(i,j) = max(& 0, \\
  & F(i, j-1) + e \text{ if } D(i,j-1) = \text{ 'left', else } F(i, j-1) + d + e , \\
  & F(i-1, j) + e \text{ if } D(i-1,j) = \text{ 'up', else } F(i-1, j) + d + e , \\
  & F(i-1,j-1) + s(A(i), B(j)))
\end{split}
$$


In [147]:
# Importing to keep track of the top 10 elements.
from heapq import heapify, heappush, heappop 
def findTop10(F):
    top10 = [(-1, (-1,-1)) for _ in range(10)]
    heapify(top10)
    for i in range(len(F)):
        for j in range(len(F[0])):
            if top10[0][0] < F[i][j]:
                heappop(top10)
                heappush(top10, (F[i][j], (i,j)))
    return list(reversed(top10)) 

In [148]:
# First read the input files to fenerate the global variables
def read_variables(parameter_file, text_file):
    with open(parameter_file, "r") as file1:
        d, e = map(int, file1.readline().strip().split(" "))
        alphabet = {key:index for (index, key) in enumerate(file1.readline().strip().split(" "))}
        scoring_matrix = []
        for i in range(len(alphabet)):
            row = list(map(int, file1.readline().strip().split(" ")))
            scoring_matrix.append(row)
    with open(text_file, "r") as file2:
        seq1 = file2.readline().strip()
        seq2 = file2.readline().strip()
    return (seq1, seq2, scoring_matrix, d, e, alphabet)

In [149]:
# Initialize the matrices, F[i][j] is the optimal alignment between seq1[0:i] and seq2[0:j]
def SWinitialize(seq1, seq2):
    seqlen1 = len(seq1)
    seqlen2 = len(seq2)
    F = [[0 for y in range(seqlen2+1)] for x in range(seqlen1+1)]
    D = [[0 for y in range(seqlen2+1)] for x in range(seqlen1+1)]
    return F, D

In [150]:
def SWinduce(F, D, seq1, seq2, scoring_matrix, d, e, alphabet):
    none, diag, up, left = 0, 1, 2, 3
    for i in range(1, len(seq1)+1):
        for j in range(1, len(seq2)+1):
            upScore = F[i-1][j] + e
            leftScore = F[i][j-1] + e
            diagScore = F[i-1][j-1] + scoring_matrix[alphabet[seq1[i-1]]][alphabet[seq2[j-1]]]
            currScore = 0 # cannot be worse
            currTrace = none # we will figure is out
            # If gap continues in the first sequence
            if D[i][j-1] == left:
                if leftScore > currScore:
                    currScore = leftScore
                    currTrace = left 
            # If initial gap in the first sequence
            else:
                if leftScore + d - e > currScore:
                    currScore = leftScore + d - e 
                    currTrace = left 
            # If gap continues in the second sequence
            if D[i-1][j] == up:
                if upScore > currScore:
                    currScore = upScore
                    currTrace = up 
            # If initial gap in the second sequence 
            else:
                if upScore + d - e > currScore:
                    currScore = upScore + d - e
                    DcurrTrace = up 
            # If no gaps
            if diagScore > currScore:
                currScore = diagScore
                currTrace = diag  
            F[i][j] = currScore 
            D[i][j] = currTrace   

In [151]:
def SWtraceback(F, D, seq1, seq2, k=10):
    none, diag, up, left = 0, 1, 2, 3
    top10 = findTop10(F)
    results = []
    for top in top10:
        score, index = top[0], top[1]
        i, j = index[0], index[1]
        last1 = i
        last2 = j
        aseq1 = ''
        aseq2 = ''
        while (D[i][j] != none):
            if D[i][j] == diag :
                aseq1 += seq1[i-1]
                aseq2 += seq2[j-1]
                i = i - 1
                j = j - 1
            if D[i][j] == up:
                aseq1 += seq1[i-1]
                aseq2 += '-'
                i = i - 1
            if D[i][j] == left:
                aseq2 += seq2[j-1]
                aseq1 += '-'
                j = j - 1
            
        results.append([score, (i, aseq1[::-1], last1), (j, aseq2[::-1], last2)])
    return(results)

In [ ]:
def calculateLambda(scoring_matrix, alphabet):
    k = 0.1
    p = 1 / len(alphabet)
    return

In [ ]:
def prettyprint(results):
    for index, result in enumerate(results):
        score = result[0]
        title = f"HSP #{index} - Score: {score} p-value: {p_val}"

In [152]:
seq1, seq2, scoring_matrix, d, e, alphabet = read_variables("parameters/DNAParams.txt", "tests/DNATest1.txt")

p = [print(item) for item in [seq1, seq2, scoring_matrix, d, e, alphabet]]
F, D = SWinitialize(seq1, seq2)
SWinduce(F, D, seq1, seq2, scoring_matrix, d, e, alphabet)
print(SWtraceback(F, D, seq1, seq2))

CGGGTAGTTAACCCTACAGCATAGAGTCGCGAGATAAAGTGCAGGAGTCTTTCGCGGCAGATTCGTACCTCAACCACGTGCTACTTTCTGGCATCACGAATCTGCCGCATAGGTCCGTGAGTCCATATGA
GGAGCACTCGAATAGTCGTGGGTACTGACTGACCCTGAGTGCCATATGA
[[1, -1, -1, -1], [-1, 1, -1, -1], [-1, -1, 1, -1], [-1, -1, -1, 1]]
-2
-1
{'A': 0, 'T': 1, 'C': 2, 'G': 3}
[[12, (114, 'CCGTGAGT-CCATATGA', 130), (32, 'CCCTGAGTGCCATATGA', 49)], [10, (114, 'CCGTGAGT-CCATAT', 128), (32, 'CCCTGAGTGCCATAT', 47)], [11, (114, 'CCGTGAGT-CCATATG', 129), (32, 'CCCTGAGTGCCATATG', 48)], [9, (129, 'A--', 130), (45, 'ATG', 48)], [9, (114, 'CCGTGAGT-CCATATG-', 129), (32, 'CCCTGAGTGCCATATGA', 49)], [8, (129, '--', 129), (45, 'AT', 47)], [9, (114, 'CCGTGAGT-CCATA', 127), (32, 'CCCTGAGTGCCATA', 46)], [8, (114, 'CCGTGAGT-CCATAT-', 128), (32, 'CCCTGAGTGCCATATG', 48)], [8, (114, 'CCGTGAGT-CCAT', 126), (32, 'CCCTGAGTGCCAT', 45)], [7, (114, 'CCGTGAGT-CCATAT--', 128), (32, 'CCCTGAGTGCCATATGA', 49)]]
